## 라이브러리 선언 및 데이터 불러오기

In [1]:
# 데이터 처리 라이브러리
import pandas as pd
import numpy as np

# 머신러닝 라이브러리
from sklearn import tree
from sklearn import ensemble

In [2]:
featuresData = pd.read_csv("../dataset/feature_regression_example.csv")

# A. 데이터 전처리

### A-1. 타입통합 / 특성 숫자컬럼 추가

In [3]:
featuresData

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155
...,...,...,...,...,...,...,...,...,...,...,...,...
100,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201648,2016,48,2412,Y,0,Y,0.421888
101,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201649,2016,49,1955,N,4,Y,0.421888
102,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201650,2016,50,1800,N,4,Y,0.352361
103,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201651,2016,51,1173,N,4,Y,0.352361


In [4]:
featuresData.dtypes

REGIONID         object
PRODUCTGROUP     object
PRODUCT          object
ITEM             object
YEARWEEK          int64
YEAR              int64
WEEK              int64
QTY               int64
HOLIDAY          object
HCLUS             int64
PROMOTION        object
PRO_PERCENT     float64
dtype: object

In [5]:
#데이터 타입 통합
featuresData["REGIONID"] = featuresData["REGIONID"].astype(str)
featuresData["PRODUCTGROUP"] = featuresData["PRODUCTGROUP"].astype(str)

In [6]:
# 특성값 숫자컬럼 변경

In [7]:
featuresData.head(1)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442


In [8]:
featuresData.PROMOTION.drop_duplicates()

0    Y
9    N
Name: PROMOTION, dtype: object

In [9]:
# 숫자가 아니라 문자를 머신러닝에 던지면 에러가 발생함.(변경 Y=1 , N=0)
featuresData["PROMOTION_LB"] = np.where(featuresData.PROMOTION =="Y",1,0)
featuresData["HOLIDAY_LB"] = np.where(featuresData.HOLIDAY =="Y",1,0)

In [10]:
# 검증
featuresData.loc[(featuresData.PROMOTION_LB == 1) &
                 (featuresData.PROMOTION == "Y") ] 

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PROMOTION_LB,HOLIDAY_LB
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442,1,1
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442,1,0
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155,1,0
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155,1,1
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201648,2016,48,2412,Y,0,Y,0.421888,1,1
101,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201649,2016,49,1955,N,4,Y,0.421888,1,0
102,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201650,2016,50,1800,N,4,Y,0.352361,1,0
103,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201651,2016,51,1173,N,4,Y,0.352361,1,0


## A-2 특성 선정 / 데이터 분리

In [11]:
# 특성 선정
corrDf = featuresData.corr() #판매량에 영향을 주는 요소들이 어떤거인지 알 수 있음.
stdCorr = 0.5

In [12]:
features = list( corrDf.loc[ (abs(corrDf.QTY) >= stdCorr) & (corrDf.QTY != 1) ].index ) ##나자신인 qty는 답안지 이기 때문에 제외시켜줘야한다.

In [13]:
features

['HCLUS', 'PRO_PERCENT', 'PROMOTION_LB', 'HOLIDAY_LB']

In [14]:
label = ["QTY"]

In [15]:
# 데이터 분리(7:3)

In [16]:
stdRatio = 0.7

In [17]:
stdIndex = int(featuresData.shape[0] * stdRatio)
stdIndex

73

In [18]:
trainingDataFeatures = featuresData.loc[0:stdIndex, features] #문제지
trainingDataLabel = featuresData.loc[0:stdIndex, label] #정답
testDataFeatures = featuresData.loc[stdIndex+1 :, features] #문제지
testDataLabel = featuresData.loc[stdIndex+1 :, label] #정답

In [19]:
print ( trainingDataFeatures.shape )
print ( trainingDataLabel.shape )
print ( testDataFeatures.shape )
print ( testDataLabel.shape )

(74, 4)
(74, 1)
(31, 4)
(31, 1)


## B 모델적용

### B-3. 모델적용

In [39]:
from sklearn import tree

In [40]:
# 알고리즘을 고정시킴으로써 예측값을 재현 가능하게 해줌(random_state=10)
dtModel = tree.DecisionTreeRegressor(random_state=10)

In [41]:
dtModel.fit(X=trainingDataFeatures,
            y=trainingDataLabel) ## 대문자 X는 문제지, 소문자 y는 정답지)

DecisionTreeRegressor(random_state=10)

## C. 예측/검증

### c-1. 예측

In [42]:
dtModel

DecisionTreeRegressor(random_state=10)

In [43]:
predictValue = dtModel.predict( X = testDataFeatures )

In [44]:
len(predictValue)

31

# 미래는 어떻게 예측할 것이냐?

In [45]:
features

['HCLUS', 'PRO_PERCENT', 'PROMOTION_LB', 'HOLIDAY_LB']

In [46]:
inputHclus = 1 # 대휴일1 소휴일 :4
inputProPercent = 0.5 # 프로모션 비율
inputPromotionLb = 1 # 프로모션 적용 
inputHlb = 1 # 홀리데이 적용 Y

In [47]:
testData = pd.DataFrame( [ [inputHclus, inputProPercent, inputPromotionLb, inputHlb] ] )

In [48]:
dtModel.predict(testData)

C:\Users\SMART27\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([2193.8])